# Phrase-structure parsing with CKY

In this assignment, we'll train a simple phrase-structure (constituency) parser on the [Penn Treebank](https://catalog.ldc.upenn.edu/ldc99t42), then implement exact inference using the CKY algorithm.

You may want to refer to the async material and lecture notes for a detailed discussion of the algorithm. The following may also be useful:
- [Guide to Penn Treebank constituent tags](http://www.surdeanu.info/mihai/teaching/ista555-fall13/readings/PennTreebankConstituents.html) - explains what `JJ`, `NNP`, and all the other tags mean.
- [Syntactic Parsing (Jurafsky & Martin, Ch. 12)](https://web.stanford.edu/~jurafsky/slp3/12.pdf) - particularly 12.1 and 12.2.
- [Accurate Unlexicalized Parsing](http://ilpubs.stanford.edu:8091/~klein/unlexicalized-parsing.pdf) (Klein & Manning, 2003)

We'll write code in three parts:
- **(a)** Initial preprocessing of the treebank
- **(b)** Calculation of production rule probabilities
- **(c)** Implementation of CKY algorithm

We provide the code for 1 and much of the framework surrounding 2 and 3.

In [1]:
# Install a few python packages using pip
from w266_common import utils
utils.require_package("tqdm")      # for progress bars
utils.require_package("graphviz")  # for rendering trees

### Preliminaries: GraphViz

**You can skip this if you have GraphViz set up.**

This notebook uses [GraphViz](https://www.graphviz.org/) to render tree structures. On Ubuntu / Debian (including Google Cloud), you can install it by running on the command line:
```
sudo apt-get install graphviz
```

For Mac OSX, you can install using Homebrew:
```
brew install graphviz
```
or see https://www.graphviz.org/download/ for more options. Run the cell below to set up rendering and show a sample tree.

In [2]:
import nltk
from w266_common import treeviz
# Monkey-patch NLTK with better Tree display that works on Cloud or other display-less server.
print("Overriding nltk.tree.Tree pretty-printing to use custom GraphViz.")
treeviz.monkey_patch(nltk.tree.Tree, node_style_fn=None, format='svg')

# Test rendering
print("Sample tree to test rendering:")
nltk.tree.Tree.fromstring("(S (NP (PRP I)) (VP (VBP love) (NNP W266)) (SYM 😄))")

In [3]:
# Import some useful libraries...
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import os, sys, collections
import copy
from importlib import reload

import numpy as np
import nltk
from nltk.tree import Tree
from IPython.display import display, HTML
from tqdm import tqdm as ProgressBar

import logging
logging.basicConfig(stream=sys.stderr, level=logging.DEBUG)

# Helpers for this assignment
from w266_common import utils, treeviz
import cky_helpers
import pcfg, pcfg_test
import cky, cky_test

# Use the sample of the Penn Treebank included with NLTK.
assert(nltk.download('treebank'))
corpus = nltk.corpus.treebank

# If you do install the full Penn Treebank, change the "False" to "True" below.
use_full_ptb = False
if use_full_ptb:
    cky_helpers.verify_ptb_install()
    corpus = nltk.corpus.ptb  # Throws errors, for some reason
    # This configures the corpus to use the WSJ section only.
    # The Brown section has some mis-bracketed trees that will cause the 
    # corpus reader to throw (many) errors.
    if not hasattr(corpus, '_parsed_sents'):
        print("Monkey-patching corpus reader...")
        corpus._parsed_sents = corpus.parsed_sents
        corpus.parsed_sents = lambda: corpus._parsed_sents(categories=['news'])

## Part (a) Exercises: API warm-up

We'll be using `nltk.tree.Tree` objects in the rest of the assignment, which provide some convenient methods for viewing and traversing parse trees, as well as extracting grammar rules (productions).

The API is documented here: http://www.nltk.org/api/nltk.html#nltk.tree.Tree

In the cells below, do the following to familiarize yourself with the Tree API:

1. Construct the tree `(S (NP foo) (VP bar))` using the Tree API. The constructor can be called as: `Tree(lhs, [rhs_1, rhs_2, ...]` where `lhs` is a string and `rhs_*` are either trees or strings.
2. Display the tree for the first sentence in the corpus. (It should be: "Pierre Vinken , 61 years old , will join the board as a nonexecutive director Nov. 29 .")
3. For the first sentence, print the label of the root node and the labels of its immediate children.
4. For the first sentence, print all the productions in the sentence. Also print the LHS of the first production, and the RHS of the second production.


In [4]:
## Part (a).1
#### YOUR CODE HERE ####
#### END(YOUR CODE) ####

In [5]:
## Part (a).2
#### YOUR CODE HERE ####
#### END(YOUR CODE) ####

(Side node: it turns out this first sentence is actually pretty meta: http://languagelog.ldc.upenn.edu/nll/?p=3594 Long live Pierre Vinken!)

In [6]:
## Part (a).3
#### YOUR CODE HERE ####
s = corpus.parsed_sents()[0]
#### END(YOUR CODE) ####

In [7]:
## Part (a).4
#### YOUR CODE HERE ####
s = corpus.parsed_sents()[0]
#### END(YOUR CODE) ####

# (a) Preprocessing

### Removing Cross-References

This first step of preprocessing takes the treebank, strips out the cross references (NPs are wrapped by special nodes that assign index numbers to them so that coreferences can be indicated).  Unfortunately, this also injects a NP-SBJ-# node between nodes you'd expect to produce one another.  Since the # changes throughout the corpus, our counts of the production rules all end up being 1 - and useless.

See NP-SBJ-1 in the tree below.  Note there is also a NP-SBJ leading to a NONE/1 subtree as a crossreference later.

In the code below we skip over nodes whose label start with NP-, connecting any children nodes to the NP-'s parent.  We also snip out any subtrees rooted by NONE.  The tree above is printed again after this next cell to illustrate the effect of this code.

### Chomsky Normal Form

Finally, CKY assumes that trees are constructed from a grammar that is in [Chomsky normal form](https://en.wikipedia.org/wiki/Chomsky_normal_form).

This means that the grammar only consists of three types of rules:
- **Binary nonterminal:** `A -> B C`
- **Unary preterminal:** `A -> a`
- **Epsilon:** `A -> `$\ \ \epsilon$

where `A`, `B`, and `C`, are non-terminals, `a` is a terminal, and $\epsilon$ is the empty sentence.

In order to accomplish this, we add new non-terminals to the language and build longer sequences of non-terminals through them.   
For example, the ternary rule
- `A -> B C D`

becomes two rules:
- `A -> B A|<C-D>`
- `A|<C-D> -> C D`

where `A|<C-D>` is a dummy symbol that we add to signifiy that it's a production of `A` that creates `C D`.

Since the resulting tree is (at most) binary, we sometimes call this process _binarization_.

#### Horizontal Markovization

The dummy-symbol system works well until you get very long grammar rules such as
- `A -> B C D E F G H I J K L`

If we followed the rule above, we'd get intermediate symbols that look like `A|<B-C-D-E-F-G-H-...>`. This would quickly lead to an explosion in the number of symbols in our grammar! Because such long productions are fairly rare, we may have trouble getting good estimates of their probability. (*Recall the sparsity problem from ngram language models!*)

One way to counter this is called _horizontal Markovization_. Similar to how in language modeling, we "forgot" all history more than (n-1) words back, we can simply choose to truncate the the history and only store shorter symbols like `A|<B-C>`, `A|<C-D>`, `A|<D-E>`, and so on. This way, we can share parameters across more examples that are similar in structure.

NLTK implements this for us in the `chomsky_normal_form` function; try changing the **`horzMarkov`** parameter below to see how it works.

Take a minute to play with the ```horzMarkov``` parameter in the block below to see how this works. 

In [8]:
sentence = corpus.parsed_sents()[35]
# Filter out NP-* nodes.
cleaned_sentence = cky_helpers.clean_tree(sentence, simplify=True)
# Convert sentence to Chomsky normal form.
cnf_sentence = copy.deepcopy(cleaned_sentence)
nltk.treetransforms.chomsky_normal_form(cnf_sentence, horzMarkov=2)
display(HTML(treeviz.render_tree(cleaned_sentence, title="Original", format='svg')))
display(HTML(treeviz.render_tree(cnf_sentence, title="Binarized (CNF)", format='svg')))

### Run Pre-Processing on Corpus

We'll loop through the whole corpus, and make copies of each sentence in CNF form. Use the `cnf_sentences` list for training the grammar in part (b).

**Note:** if you're using the `treebank` corpus sample, this should run in just a few seconds. But if you use the full Penn Treebank, it'll take around 1-2 minutes to process all the trees. If you get an error "`AttributeError: 'tqdm' object has no attribute 'miniters'`", ignore it - the code should still work.

In [9]:
# Preprocess the treebank.
cleaned_sentences = []
cnf_sentences = []
for sentence in ProgressBar(corpus.parsed_sents(), desc="Processing sentences"):
    # Filter out NP-* nodes.
    cleaned_sentence = cky_helpers.clean_tree(sentence, simplify=True)
    cleaned_sentences.append(cleaned_sentence)
    
    # Convert sentence to Chomsky normal form.
    cnf_sentence = copy.deepcopy(cleaned_sentence)
    nltk.treetransforms.chomsky_normal_form(cnf_sentence, horzMarkov=2)
    cnf_sentences.append(cnf_sentence)

# (b) Production rule probabilities

In this next section, you'll compute about production rule probabilities.

We won't use epsilon rules, so all of our rules will be of the form:
- Binary nonterminal: `A -> B C`
- Unary preterminal: `A -> a`

The left hand side (LHS) of these rules only ever consist of a single nonterminal.  The right hand side (RHS) consists of either two non-terminals or one terminal.

We'll do this in two stages:
- Count LHS, and (LHS,RHS) each in their own dict
- Calculate $ P(RHS | LHS) = \frac{count(LHS, RHS)}{count(LHS)} $

## Part (b) Implementation: Training the grammar

Now that you're comfortable with NLTK Tree objects, let's use them to build our grammar. We've implemented a skeleton in **`pcfg.py`**; your job is to finish the implementation of the `pcfg.PCFG` class. Specifically:

- Implement `update_counts`, which updates the production counts for a single sentence.
- Implement `compute_scores`, which computes log-probabilities.

Read the documentation in `pcfg.py` for the names of the data structures you should populate, and their precise types. Both functions should be straightforward, and only require a couple of lines of code each!

#### Indexing Rules

Your code here need only deal with straightforward maps of productions, but in order to parse efficiently we need to build an inverted index, keyed on the rule's RHS. This way, we can quickly look up rules (and their scores) that would combine two subtrees during the CKY algorithm.

We've implemented this for you in `pcfg.PCFG.build_index()`, but you'll want to look carefully at how that function works - when you implement CKY in part(c), you'll make heavy use of the `grammar.parsing_index` structure.

### Testing `update_counts()`

If everything works, you should see this in the cell below:

```
Top productions:
(PP -> IN NP, 7369)
(, -> ',', 4885)
(DT -> 'the', 4038)
(. -> '.', 3828)
(S|<VP-.> -> VP ., 3071)
(NP -> NP PP, 2644)
(S -> VP, 2335)
(IN -> 'of', 2319)
(TO -> 'to', 2161)
(NP -> DT NN, 2020)
```

### Testing `compute_scores()`

If everything went well, you should see:
```
food [(NN, -6.7128043057880404)]
a [(DT, -1.4717815426061982), (JJ, -7.9783109698677208), (IN, -9.1959371416654392), (LS, -2.5649493574615367)]
I [(NNP, -8.4563810520194806), (PRP, -2.7203634613355669)]
```

In [10]:
reload(pcfg)
utils.run_tests(pcfg_test, ["TestPCFG"])

In [11]:
reload(pcfg)

grammar = pcfg.PCFG()
for sentence in ProgressBar(cnf_sentences, desc="Counting productions"):
    grammar.update_counts(sentence)
    
print("Top productions:")
for p in grammar.top_productions():  # Top productions, by un-normalized count
    print(p)
print("")

grammar.compute_scores()  # compute log-probabilities
grammar.build_index()     # prepare for parsing

for w in ['food', 'a', 'I']:
    print(w, grammar.parsing_index[(w,)])

You don't need to do anything with this next cell except to run it.

It's not particularly useful, but if you need to keep track of what each variable contains, this provides a useful reference.

In [12]:
print('Productions (nltk.grammar.Production):')
for (production, count) in grammar.production_counts.most_common(5):
    print (production, count)

print('\n\nLHS counts:')
for (lhs, count) in grammar.lhs_counts.most_common(5):
    print((lhs, count))
    
print('\n\nLog Probabilities:')
print('\n'.join([str(x) for x in grammar.parsing_index.items()][0:10]))

# (c) Implement CKY

After that bit of preamble, you only have one more task to go!  It's a big one though, so do take your time and get things right. 

**In `cky.py`, implement the `CKY_apply_preterminal_rules` and `CKY_apply_binary_rules` functions.**

We've set up the skeleton of the CKY algorithm for you in the `CKY` function; be sure to read the in-line comments there carefully before you start. The outline is:
1. Construct the chart (`make_chart`, provided)
2. Populate the first row of the chart using preterminal rules (`CKY_apply_preterminal_rules`)
3. Populate the rest of the chart using binary rules (`CKY_apply_binary_rules`)
4. Read off the top cell for the final derivation. (provided)

We'll implement the chart itself as a dict that you can index into first by cell position and then by non-terminal like this:
```
chart[(0, 1)][NN]
```

The value is an [nltk.tree.ProbabilisticTree](http://www.nltk.org/api/nltk.html#nltk.tree.ProbabilisticTree), which is just like an `nltk.tree.Tree` except that it has an additional `logprob()` method that returns the score (log-probability). Similarly, the constructor takes an additional argument: `pt = ProbabilisticTree(lhs, (rhs_1,...), logprob=score)` - you should use this to construct the backtrace trees.

See the in-line comments in `cky.py` for additional hints and advice.

In [13]:
reload(cky)
utils.run_tests(cky_test, ["TestParsing"])

In [14]:
reload(cky)
parser = cky.CKYParser(grammar)
derivation = parser.parse('I eat red hot food with a knife'.split(), 'S')

assert round(derivation.logprob(), 2) == -64.08
derivation

### Experimentation

Try a few more sentences.  Do you notice any patterns with your results?  Any common types of errors?  Are these an artifact of CKY, or of how you did the markovization/counting?

Put any code you write in the next cell and a writeup of the results in the cell after.

(If you have a format that's more natural to your description, feel free to deviate from this format.)

In [15]:
#### YOUR CODE HERE ####

parser.parse('I eat some food'.split(), 'S')

#### END(YOUR CODE) ####

#### Your observation title 1
Your observation detail.  (Copy and paste these two lines and fill them in for each observation.)